# Excited States via VQD

This notebook validates that:
- VQD runs for `num_states=k` states
- Works under noise

It prints:
- Lowest $k$ exact eigenvalues of $H$
- VQD final energies for states $0, 1, ..., k-1$
- $|ΔE_i|$ versus exact eigenvalue i

In [ ]:
import numpy as np
import pennylane as qml

from vqe.hamiltonian import build_hamiltonian
from vqe.vqd import run_vqd

## Configuration

In [ ]:
molecule = "H2"

# k-state target
num_states = 4

# Ansatz/optimizer
ansatz_name = "StronglyEntanglingLayers"
optimizer_name = "Adam"
steps = 250
stepsize = 0.2
seed = 0

# VQD deflation schedule
beta = 10.0
beta_start = 0.0
beta_ramp = "cosine"
beta_hold_fraction = 0.1

# Noise knobs (leave at 0 for noiseless section)
depolarizing_prob = 0.00
amplitude_damping_prob = 0.00


## Build Hamiltonian + exact spectrum benchmark

In [ ]:
H, n_wires, symbols, coordinates, basis = build_hamiltonian(molecule)
Hmat = np.array(qml.matrix(H), dtype=float)
evals = np.sort(np.linalg.eigvalsh(Hmat))

print(f"Molecule: {molecule}")
print(f"Qubits:   {n_wires}")
print(f"Basis:    {basis}")
print(f"Lowest {max(10, num_states)} exact eigenvalues (Ha):")
for i, e in enumerate(evals[: max(10, num_states)]):
    print(f"#{i:>2}: {float(e): .10f}")

Molecule: H2
Qubits:   4
Basis:    STO-3G

Lowest 10 exact eigenvalues (Ha):
# 0: -1.1372701749
# 1: -0.5387095807
# 2: -0.5387095807
# 3: -0.5324790143
# 4: -0.5324790143
# 5: -0.5324790143
# 6: -0.4469857253
# 7: -0.4469857253
# 8: -0.1699013991
# 9:  0.2378052722


/tmp/ipykernel_5583/32052762.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  Hmat = np.array(qml.matrix(H), dtype=float)


## Table summary helper

In [ ]:
def _summarize_vqd(result: dict, exact_evals: np.ndarray, *, label: str) -> None:
    energies = result["energies_per_state"]
    finals = [float(traj[-1]) for traj in energies]

    print("\n" + "=" * 80)
    print(label)
    print("=" * 80)

    for i, Ei in enumerate(finals):
        target = float(exact_evals[i]) if i < len(exact_evals) else float("nan")
        err = abs(Ei - target) if np.isfinite(target) else float("nan")
        print(f"State {i}: E_final = {Ei:+.10f} Ha   |ΔE_i| vs exact[{i}] = {err:.3e} Ha")


## 1) Noiseless k-state VQD

In [ ]:
noiseless_results = run_vqd(
    molecule=molecule,
    num_states=num_states,
    beta=beta,
    beta_start=beta_start,
    beta_ramp=beta_ramp,
    beta_hold_fraction=beta_hold_fraction,
    ansatz_name=ansatz_name,
    optimizer_name=optimizer_name,
    steps=steps,
    stepsize=stepsize,
    seed=seed,
    noisy=False,
    depolarizing_prob=0.0,
    amplitude_damping_prob=0.0,
    noise_model=None,
    plot=True,
    force=False,
)

_summarize_vqd(noiseless_results, evals, label="Noiseless VQD")

📂 Using cached VQD result: /workspaces/Variational_Quantum_Eigensolver/results/vqe/H2__VQD(StronglyEntanglingLayers)_4states__Adam__VQD__noiseless__s0__08400c29e096.json

Noiseless VQD
State 0: E_final = -1.1372701749 Ha   |ΔE_i| vs exact[0] = 2.995e-12 Ha
State 1: E_final = -0.5324790143 Ha   |ΔE_i| vs exact[1] = 6.231e-03 Ha
State 2: E_final = -0.5324802445 Ha   |ΔE_i| vs exact[2] = 6.229e-03 Ha
State 3: E_final = -0.5387095305 Ha   |ΔE_i| vs exact[3] = 6.231e-03 Ha


## 2) Noisy VQD using built-in depolarizing/amplitude probs

In [ ]:
depolarizing_prob = 0.05
amplitude_damping_prob = 0.05

noisy_results = run_vqd(
    molecule=molecule,
    num_states=num_states,
    beta=beta,
    beta_start=beta_start,
    beta_ramp=beta_ramp,
    beta_hold_fraction=beta_hold_fraction,
    ansatz_name=ansatz_name,
    optimizer_name=optimizer_name,
    steps=steps,
    stepsize=stepsize,
    seed=seed,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    noise_model=None,
    plot=True,
    force=False,
)

_summarize_vqd(
    noisy_results,
    evals,
    label=f"Noisy VQD: dep={depolarizing_prob}, amp={amplitude_damping_prob}",
)

📂 Using cached VQD result: /workspaces/Variational_Quantum_Eigensolver/results/vqe/H2__VQD(StronglyEntanglingLayers)_4states__Adam__VQD__noisy__s0__dc6de38a5282.json

Noisy VQD (built-in noise): dep=0.05, amp=0.05
State 0: E_final = -0.9915488650 Ha   |ΔE_i| vs exact[0] = 1.457e-01 Ha
State 1: E_final = -0.4426318144 Ha   |ΔE_i| vs exact[1] = 9.608e-02 Ha
State 2: E_final = -0.4242450193 Ha   |ΔE_i| vs exact[2] = 1.145e-01 Ha
State 3: E_final = -0.4501697172 Ha   |ΔE_i| vs exact[3] = 8.231e-02 Ha
